In [ ]:
#|default_exp utils

# debuggable utils
build useful functions along the way for experiments

## Find out everything about a module

In [ ]:
#|export
from inspect import getmembers, isfunction, isclass, isbuiltin, getsource
import os.path, pkgutil
from pprint import pprint
import re
import ast
import inspect
from fastcore.meta import *
from fastcore.imports import *
from inspect import _signature_from_callable

In [ ]:
import fastcore.meta as fm
import fastai.basics as fb
from nbdev.showdoc import *

In [ ]:
#|export
def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here
               dun:bool=False, # print all items in __all__
               func:bool=False, # print all user defined functions
               clas:bool=False, # print all class objects
               bltin:bool=False, # print all builtin funcs or methods
               lib:bool=False, # print all the modules of the library it belongs to
               cal:bool=False # print all callables
             ): 
    'Check what inside a module: __all__, functions, classes, builtins, and callables'
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0
    funcs = getmembers(mo, isfunction)
    classes = getmembers(mo, isclass)
    builtins = getmembers(mo, isbuiltin)
    callables = getmembers(mo, callable)
    pkgpath = os.path.dirname(mo.__file__)
    print(f"{mo.__name__} has: \n{dun_all} items in its __all__, and \n{len(funcs)} user defined functions, \n{len(classes)} classes or class objects, \n{len(builtins)} builtin funcs and methods, and\n{len(callables)} callables.\n")  
    if hasattr(mo, "__all__") and dun: pprint(mo.__all__)
    if func: 
        print(f'The user defined functions are:')
        pprint([i[0] for i in funcs])
    if clas: 
        print(f'The class objects are:')
        pprint([i[0] for i in classes])
    if bltin: 
        print(f'The builtin functions or methods are:')
        pprint([i[0] for i in builtins])
    if cal: 
        print(f'The callables are: ')
        pprint([i[0] for i in callables])
    if lib: 
        modules = [name for _, name, _ in pkgutil.iter_modules([pkgpath])]
        print(f'The library has {len(modules)} modules')
        pprint(modules)

```python

def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here
               dun:bool=False, # print all items in __all__
               func:bool=False, # print all user defined functions
               clas:bool=False, # print all class objects
               bltin:bool=False, # print all builtin funcs or methods
               lib:bool=False, # print all the modules of the library it belongs to
               cal:bool=False # print all callables
             ): 
    'Check what inside a module: __all__, functions, classes, builtins, and callables'
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0
    funcs = getmembers(mo, isfunction)
    classes = getmembers(mo, isclass)
    builtins = getmembers(mo, isbuiltin)
    callables = getmembers(mo, callable)
    pkgpath = os.path.dirname(mo.__file__)
    print(f"{mo.__name__} has: \n{dun_all} items in its __all__, and \n{len(funcs)} user defined functions, \n{len(classes)} classes or class objects, \n{len(builtins)} builtin funcs and methods, and\n{len(callables)} callables.\n")  
    if hasattr(mo, "__all__") and dun: pprint(mo.__all__)
    if func: 
        print(f'The user defined functions are:')
        pprint([i[0] for i in funcs])
    if clas: 
        print(f'The class objects are:')
        pprint([i[0] for i in classes])
    if bltin: 
        print(f'The builtin functions or methods are:')
        pprint([i[0] for i in builtins])
    if cal: 
        print(f'The callables are: ')
        pprint([i[0] for i in callables])
    if lib: 
        modules = [name for _, name, _ in pkgutil.iter_modules([pkgpath])]
        print(f'The library has {len(modules)} modules')
        pprint(modules)
```

In [ ]:
whatinside(fm, func=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

The user defined functions are:
['_funcs_kwargs',
 '_mk_param',
 '_rm_self',
 'all_equal',
 'anno_dict',
 'any_is_instance',
 'array_equal',
 'contextmanager',
 'copy',
 'delegates',
 'df_equal',
 'empty2none',
 'equals',
 'funcs_kwargs',
 'in_colab',
 'in_ipython',
 'in_jupyter',
 'in_notebook',
 'ipython_shell',
 'is_close',
 'is_coll',
 'is_iter',
 'isinstance_str',
 'method',
 'nequals',
 'noop',
 'noops',
 'remove_prefix',
 'remove_suffix',
 'test',
 'test_close',
 'test_eq',
 'test_eq_type',
 'test_fail',
 'test_fig_exists',
 'test_is',
 'test_ne',
 'test_shuffled',
 'test_sig',
 'test_stdout',
 'test_warns',
 'use_kwargs',
 'use_kwargs_dict']


In [ ]:
whatinside(fb, bltin=True)

fastai.basics has: 
0 items in its __all__, and 
430 user defined functions, 
237 classes or class objects, 
4 builtin funcs and methods, and
696 callables.

The builtin functions or methods are:
['array', 'as_tensor', 'reduce', 'warn']


In [ ]:
whatinside(fb, lib=True)

fastai.basics has: 
0 items in its __all__, and 
430 user defined functions, 
237 classes or class objects, 
4 builtin funcs and methods, and
696 callables.

The library has 24 modules
['_modidx',
 '_nbdev',
 '_pytorch_doc',
 'basics',
 'callback',
 'collab',
 'data',
 'distributed',
 'fp16_utils',
 'imports',
 'interpret',
 'layers',
 'learner',
 'losses',
 'medical',
 'metrics',
 'optimizer',
 'tabular',
 'test_utils',
 'text',
 'torch_basics',
 'torch_core',
 'torch_imports',
 'vision']


## Find out the version of a library

### `whichversion` source

In [ ]:
#|export
from importlib.metadata import version, metadata, distribution
from platform import python_version 

In [ ]:
#|export
def whichversion(libname:str, # library name
                req:bool=False, # print lib requirements 
                file:bool=False): # print all lib files
    "Give you library version and other basic info."
    if libname=='python':
        print(f"python: {python_version()}")
    else: 
        print(f"{metadata(libname)['Name']}: {version(libname)} \n{metadata(libname)['Summary']}\
    \n{metadata(libname)['Author']} \n{metadata(libname)['Home-page']} \
    \npython_version: {metadata(libname)['Requires-Python']} \
    \n{distribution(libname).locate_file(libname)}")

    if req: 
        print(f"\n{libname} requires: ")
        pprint(distribution(libname).requires)
    if file: 
        print(f"\n{libname} has: ")
        pprint(distribution(libname).files)
    

In [ ]:
pprint(getsource(whichversion))

('def whichversion(libname:str, # library name\n'
 '                req:bool=False, # print lib requirements \n'
 '                file:bool=False): # print all lib files\n'
 '    "Give you library version and other basic info."\n'
 "    if libname=='python':\n"
 '        print(f"python: {python_version()}")\n'
 '    else: \n'
 '        print(f"{metadata(libname)[\'Name\']}: {version(libname)} '
 "\\n{metadata(libname)['Summary']}\\\n"
 "    \\n{metadata(libname)['Author']} \\n{metadata(libname)['Home-page']} \\\n"
 "    \\npython_version: {metadata(libname)['Requires-Python']} \\\n"
 '    \\n{distribution(libname).locate_file(libname)}")\n'
 '\n'
 '    if req: \n'
 '        print(f"\\n{libname} requires: ")\n'
 '        pprint(distribution(libname).requires)\n'
 '    if file: \n'
 '        print(f"\\n{libname} has: ")\n'
 '        pprint(distribution(libname).files)\n')


```python
from importlib.metadata import version, metadata, distribution
from platform import python_version 

def whichversion(libname:str, # library name
                req:bool=False, # print lib requirements 
                file:bool=False): # print all lib files
    "Give you library version and other basic info."
    if libname=='python':
        print(f"python: {python_version()}")
    else: 
        print(f"{metadata(libname)['Name']}: {version(libname)} \n{metadata(libname)['Summary']}\
    \n{metadata(libname)['Author']} \n{metadata(libname)['Home-page']} \
    \npython_version: {metadata(libname)['Requires-Python']} \
    \n{distribution(libname).locate_file(libname)}")

    if req: 
        print(f"\n{libname} requires: ")
        pprint(distribution(libname).requires)
    if file: 
        print(f"\n{libname} has: ")
        pprint(distribution(libname).files)
```

#### How to break a line in python code

see the code above

### `whichversion` examples

In [ ]:
whichversion('python')

python: 3.9.13


In [ ]:
whichversion('fastcore')

fastcore: 1.5.22 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
whichversion('nbdev')

nbdev: 2.2.6 
Create delightful software with Jupyter Notebooks    
Jeremy Howard and Hamel Husain 
https://github.com/fastai/nbdev     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/nbdev


In [ ]:
whichversion('nbdev', req=True)

nbdev: 2.2.6 
Create delightful software with Jupyter Notebooks    
Jeremy Howard and Hamel Husain 
https://github.com/fastai/nbdev     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/nbdev

nbdev requires: 
['fastcore (>=1.5.19)',
 'execnb (>=0.0.10)',
 'astunparse',
 'ghapi',
 'PyYAML',
 'asttokens ; python_version=="3.7"',
 "nbdev-numpy ; extra == 'dev'",
 "nbdev-stdlib ; extra == 'dev'",
 "pandas ; extra == 'dev'",
 "matplotlib ; extra == 'dev'",
 "black ; extra == 'dev'"]


In [ ]:
whichversion('nbdev', file=True)

nbdev: 2.2.6 
Create delightful software with Jupyter Notebooks    
Jeremy Howard and Hamel Husain 
https://github.com/fastai/nbdev     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/nbdev

nbdev has: 
[PackagePath('../../../bin/nbdev_bump_version'),
 PackagePath('../../../bin/nbdev_changelog'),
 PackagePath('../../../bin/nbdev_clean'),
 PackagePath('../../../bin/nbdev_conda'),
 PackagePath('../../../bin/nbdev_create_config'),
 PackagePath('../../../bin/nbdev_deploy'),
 PackagePath('../../../bin/nbdev_docs'),
 PackagePath('../../../bin/nbdev_export'),
 PackagePath('../../../bin/nbdev_filter'),
 PackagePath('../../../bin/nbdev_fix'),
 PackagePath('../../../bin/nbdev_help'),
 PackagePath('../../../bin/nbdev_install'),
 PackagePath('../../../bin/nbdev_install_hooks'),
 PackagePath('../../../bin/nbdev_install_quarto'),
 PackagePath('../../../bin/nbdev_merge'),
 PackagePath('../../../bin/nbdev_migrate'),
 PackagePath('../../../bin/nbdev_new'),
 PackagePath

## print out info during debugging

Nice [tutorial](https://realpython.com/python-eval-function/) on how to use `eval`

In [ ]:
eval?

Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


### dbprint to display evalutaion

In [ ]:
def dbprint(name:str, *code, **env):
    print(f"\n{name}:==============================")
    for c in code:
        if "=" in c:
            exec(c)
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        else: 
            print(f"{c}: {eval(c, {}, {**env})}")

In [ ]:
def low(a, b=1): pass
def mid(c, d=1, **kwargs): pass
from_f = mid
to_f = low

In [ ]:
dbprint("getattr(from_f,'__func__',from_f)", "k = hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)


getattr(from_f,'__func__',from_f):==============================
k = hasattr(from_f, '__func__') => k : False
hasattr(to_f, '__func__'): False
to_f: <function low>
from_f: <function mid>


In [ ]:
def test():
    def t1(): pass
    def t2(): pass
    from_f = t1
    to_f = t2
    dbprint("getattr(from_f,'__func__',from_f)", "hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)

test()


getattr(from_f,'__func__',from_f):==============================
hasattr(from_f, '__func__'): False
hasattr(to_f, '__func__'): False
to_f: <function test.<locals>.t2>
from_f: <function test.<locals>.t1>


### dbprint to handle assignment too

In [ ]:
"=" in "a = 1"

True

In [ ]:
from math import *
exec("print(dir())", globals())


['AutoInit', 'BasicHtmlRenderer', 'BasicMarkdownRenderer', 'BuiltinFunctionType', 'BuiltinMethodType', 'BypassNewMeta', 'Dict', 'DocmentTbl', 'FixSigMeta', 'FunctionType', 'Generator', 'IN_COLAB', 'IN_IPYTHON', 'IN_JUPYTER', 'IN_NOTEBOOK', 'In', 'Iterable', 'Iterator', 'List', 'MethodDescriptorType', 'MethodType', 'MethodWrapperType', 'NewChkMeta', 'NoneType', 'Optional', 'Out', 'Path', 'PrePostInitMeta', 'Sequence', 'Set', 'ShowDocRenderer', 'SimpleNamespace', 'Tuple', 'Union', 'WrapperDescriptorType', '_', '_22', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', '_signature_from_callable', 'acos', 'acosh', 'all_equal', 'anno_dict', 'any_is_instance', 'array_equal', 'asin', 'asinh', 'ast', 'atan', 'atan2', 'atanh',

In [ ]:
from math import *
exec("print(dir())", {}, {"sum": sum, "print": print, "dir": dir, "c": 1})


['c', 'dir', 'print', 'sum']


In [ ]:
from math import *
exec("print(dir())", {"__builtins__" : __builtin__}, {"sum": sum, "print": print, "dir": dir, "c": 1})

['c', 'dir', 'print', 'sum']


In [ ]:
from math import *
exec("print(dir())", globals(), {"sum": sum, "print": print, "dir": dir, "c": 1})


['c', 'dir', 'print', 'sum']


In [ ]:
exec("e = 2 + 4")
eval("e")

6

In [ ]:
exec("c = 2 + 9", globals())
eval("c")

11

In [ ]:
exec("d = 2 + 10", globals(), locals())
eval("d")

12

In [ ]:
exec("z = 2 + g", globals(), locals().update({'g':3}))
eval("z")

5

In [ ]:
exec("c = 2 + f", globals(), {'f':3})
eval("c")

11

In [ ]:
exec("c = 2 + f", globals(), locals().update({'f':3}))
eval("c")

5

In [ ]:
exec("c = 2 + f", globals().update({'f':4}), locals())
eval("c")

6

In [ ]:
exec("print (dir())",{})

['__builtins__']


In [ ]:
exec ("print (dir())",{}, {'a':1, 'b':2})

['a', 'b']


In [ ]:
exec ("print (dir())", {'c':3}, {'a':1, 'b':2})

['a', 'b']


In [ ]:
'f' in locals()

True

In [ ]:
locals().update({'f':1})

In [ ]:
'f' in locals()

True

### The simple working version

In [ ]:
def dbprint(name:str, *code, **env):
    print(f"\n{name}:==============================")
    for c in code:
        if "=" in c: # updated to handle assignment
            exec(c, globals(), locals().update(env)) 
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        else: 
            print(f"{c}: {eval(c, {}, {**env})}") # working version

In [ ]:
def dbprint(name:str, *code, **env):
    print(f"\n{name}:==============================")
    for c in code:
        if "=" in c: # updated to handle assignment
            exec(c, globals(), locals().update(env)) 
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        else: 
            print(f"{c}: {eval(c, globals(), locals().update(env))}") # problematic
            # print(f"{c}: {eval(c, globals().update(env), locals().update(env))}")

In [ ]:
a = 10
string = "string"
dbprint("test assignment with exec and eval", "k = 1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
k = 1 + 2 + a => k : 13


In [ ]:
a = 10
string = "string"
dbprint("test assignment with exec and eval", "1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
1 + 2 + a: 13


In [ ]:
def test():
    def t1(): pass
    def t2(): pass
    from_f = t1
    to_f = t2
    dbprint("getattr(from_f,'__func__',from_f)", "hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)

test()


getattr(from_f,'__func__',from_f):==============================
hasattr(from_f, '__func__'): False
hasattr(to_f, '__func__'): False
to_f: <function low>
from_f: <function mid>


In [ ]:
def low(a, b=1): pass
def mid(c, d=1, **kwargs): pass
from_f = mid
to_f = low

In [ ]:
dbprint("getattr(from_f,'__func__',from_f)", "k = hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)


getattr(from_f,'__func__',from_f):==============================
k = hasattr(from_f, '__func__') => k : False
hasattr(to_f, '__func__'): False
to_f: <function low>
from_f: <function mid>


### A working version but slightly longer

In [ ]:
def dbprint(name:str, *code, **env):
    print(f"\n{name}:==============================")
    for c in code:
        if "=" in c: # updated to handle assignment
            exec(c, globals(), locals().update(env)) 
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        else: 
            # print(f"{c}: {eval(c, globals().update(env), locals().update(env))}") # working too
            print(f"{c}: {eval(c, globals().update(env))}") # working too

In [ ]:
a = 10
string = "string"
dbprint("test assignment with exec and eval", "k = 1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
k = 1 + 2 + a => k : 13


In [ ]:
a = 10
string = "string"
dbprint("test assignment with exec and eval", "1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
1 + 2 + a: 13


In [ ]:
def test():
    def t1(): pass
    def t2(): pass
    from_f = t1
    to_f = t2
    dbprint("getattr(from_f,'__func__',from_f)", "hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)

test()


getattr(from_f,'__func__',from_f):==============================
hasattr(from_f, '__func__'): False
hasattr(to_f, '__func__'): False
to_f: <function test.<locals>.t2>
from_f: <function test.<locals>.t1>


In [ ]:
def low(a, b=1): pass
def mid(c, d=1, **kwargs): pass
from_f = mid
to_f = low

In [ ]:
dbprint("getattr(from_f,'__func__',from_f)", "k = hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)


getattr(from_f,'__func__',from_f):==============================
k = hasattr(from_f, '__func__') => k : False
hasattr(to_f, '__func__'): False
to_f: <function low>
from_f: <function mid>


### exec and eval's locals has priority over global

In [ ]:
'x' in globals()

False

In [ ]:
'x' in locals()

False

In [ ]:
globals()['x'] = 0
globals()['x']

0

In [ ]:
locals()['x'] = 1
locals()['x']

1

In [ ]:
globals()['x']

1

In [ ]:
exec("x = x + 1", globals())
eval("x")

2

In [ ]:
exec("x = x + 1", globals(), locals())
eval("x")

3

In [ ]:
exec("x = x + 1", globals(), locals().update({'x':10}))
eval("x")

11

In [ ]:
exec("x = x + 1", globals().update({'x':20}), locals().update({'x':10}))
eval("x")

11

In [ ]:
exec("x = x + 1", globals().update({'x':20}))
eval("x")

21

In [ ]:
def dbprint(name:str, *code, **env):
    print(f"\n{name}:==============================")
    for c in code:
        if "=" in c: # updated to handle assignment
            exec(c, globals(), locals().update(env)) 
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        else: 
            # print(f"{c}: {eval(c, globals().update(env), locals().update(env))}") # working too
            # print(f"{c}: {eval(c, globals().update(env))}") # working too
            print(f"{c}: {eval(c, globals(), locals().update(env))}") # working too

In [ ]:
a = 10
string = "string"
dbprint("test assignment with exec and eval", "k = 1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
k = 1 + 2 + a => k : 13


In [ ]:
a = 5
string = "string"
dbprint("test assignment with exec and eval", "1 + 2 + a", a = a, string=string)


test assignment with exec and eval:==============================
1 + 2 + a: 8


In [ ]:
globals()['from_f']

<function __main__.mid(c, d=1, **kwargs)>

In [ ]:
locals()['from_f']

<function __main__.mid(c, d=1, **kwargs)>

In [ ]:
def t1(): pass
def t2(): pass
from_f = t1
to_f = t2
dbprint("getattr(from_f,'__func__',from_f)", "hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=t1, to_f=t2)



getattr(from_f,'__func__',from_f):==============================
hasattr(from_f, '__func__'): False
hasattr(to_f, '__func__'): False
to_f: <function t2>
from_f: <function t1>


In [ ]:
def low(a, b=1): pass
def mid(c, d=1, **kwargs): pass
from_f = mid
to_f = low

In [ ]:
dbprint("getattr(from_f,'__func__',from_f)", "k = hasattr(from_f, '__func__')", "hasattr(to_f, '__func__')", "to_f", "from_f", from_f=from_f, to_f=to_f)


getattr(from_f,'__func__',from_f):==============================
k = hasattr(from_f, '__func__') => k : False
hasattr(to_f, '__func__'): False
to_f: <function low>
from_f: <function mid>


### A trial and error version 

All the versions above seem working for the tests above, but the following one works for the real debugging codes

In [ ]:
def dbprint(name:str, *code, **env): 
    print(f"\n{name}:==============================")
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if "=" in c: 
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition("=")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        
        # handle evaluation
        else: 
            print(f"{c}: {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time


### to differentiate between assignment and != and ==

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func."
    
    print(f"\n{src}:==============================")
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
        
        # handle evaluation
        else: 
            print(f"{c}: {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time


In [ ]:
"k.__delopt__ = {...}".partition(" = ")[0]

'k.__delopt__'

In [ ]:
dbprint("tests = and != in the same line", "a = 5 == 4")


tests = and != in the same line:==============================
a = 5 == 4 => a: False


In [ ]:
dbprint("tests = and != in the same line", "a = 5 != 4")


tests = and != in the same line:==============================
a = 5 != 4 => a: True


### handle if statement

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func."
    
    print(f"\n{src}:==============================")
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            if eval(cond):
                print(f"{c} => ")
                exec(c, globals().update(env))
        
        # handle evaluation
        else: 
            print(f"{c}: {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time


In [ ]:
x = 100
exec("if x: print(x)")

100


In [ ]:
import re

c = 'if abc == def: print()'

cond = re.search('if (.*?):', c).group(1)
cond

'abc == def'

### Handle for in loop and in-block printing

In [ ]:
#|export
def checksource():
    lst = defaults.src.split('\n')
    for l in lst: 

        if l.strip() in defaults.deb:
            print('{:<157}'.format(l))
        else: 
            print('{:=<157}'.format(l))
    defaults.deb = None # make sure defaults.deb set to None for later debugging srcode.

In [ ]:
#|export
defaults = type('defaults', (object,), {'block': False, # whether inside a block of code investigation or not
                                     'src': None, # store the source code of the functiong being debugged
                                     'deb': None, # store the debuggable source code
                                     'debp': None, # store the debuggable source code for color printing
                                     'name': None, # the name of the func to be debugged
                                     'startsrc': None, # a piece of str in the starting line of the src code
                                     'endsrc': None, # a piece of str in the ending line of the src code
                                     'eg': None, # save an example in str
                                     'margin': 157, # for align to the right
                                     'multi': False, # debugging multiline source codes
                                     'srcdbps': [], # a list to store srclines and dbcodes
                                     'src2dbp': type('fastcore.meta', (object,), {'delegates': [], # a list of lists of (srcline, dbcode) 
                                                                                  'delegatesdb': None, # the debuggable source
                                                                                  'FixSigMeta': [], # a list of lists of (srcline, dbcode) 
                                                                                  'FixSigMetadb': None, # the debuggable source
                                                                                  '_signature_from_callable': [], # a list of lists of (srcline, dbcode) 
                                                                                  '_signature_from_callabledb': None, # the debuggable source
                                                                                 }) # store a list of (srcline, dbcode)
                                    }) 

In [ ]:
defaults.src2dbp.delegates.append(('ab', 'cd'))

In [ ]:
defaults.src2dbp.delegates

[('ab', 'cd')]

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure the include all the necessary env variables to avoid \
    the same variable with different values from different scopes."
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print(f"\n{src}:===inside a block===") # inside a block
    else:
        print(f"\n{src}:================================================================================") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    c1 = c + " print(f'{key}:{eval(key)}, {value}:{eval(value)}')" 
                    print(f"{c} => ")          
                    exec(c1, globals().update(env))
                
                else:
                    c1 = c + " print(f'item:{variables}')"
                    print(f"{c} => ")          
                    exec(c1, globals().update(env))
                    

            else: 
                print(f"{c} => ")          
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} => {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### clean up the format for readability

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure the include all the necessary env variables to avoid \
    the same variable with different values from different scopes."
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print(f"\n{src}:===inside a block===") # inside a block
    else:
        print(f"\n{src}:================================================================================") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### Add a few more caution messages for avoiding unnecessary errors in using dbprint

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print(f"\n{src}:===inside a block===") # inside a block
    else:
        print(f"\n{src}:================================================================================") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### Make the output more distinguishable between in and out of code block

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('{:>157}'.format("===inside a block==="))
    else:
        print('{:>157}'.format("==================================================="))
    
    print(f"\n{src} <===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle assignment
        if " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

In [ ]:
src = "this is a piece of code"
# 157
endline = '{:>157}'.format("====================")

print(endline)
# print(f"\n{src} =====> {endline}") 


In [ ]:

text = '{0:{fill}{align}155}'.format("a line of code", fill='<', align='<')
print(text)

a line of code<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


### adding features to handle a block of code execution

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('{:>157}'.format("===inside a block==="))
    else:
        print('{:>157}'.format("==================================================="))
    
    # print out markers for a block of source and a line source code 
    if "\n" in src: 
        print(src)
        print('{:^157}'.format("================== a block of source codes ====================="))
    else:
        print(src + "<===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle a block of code
        if "\n" in c: 
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

In [ ]:
import ast
c ="""
a = 1
b = 2
c = a + b
"""
block = ast.parse(c, mode='exec')
exec(compile(block, '<string>', mode='exec'))

In [ ]:
b

2

### add feature to print out source code in both line and block to the right end of the page

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('{:>157}'.format("===inside a block==="))
        print('{:>157}'.format(src))

    else:
        print('{:>157}'.format("==================================================="))
        print('{:>157}'.format(src))        
    
    # print out markers for a block of source and a line source code 
    if "\n" in src: 
        # print(src)
        # print('{:^157}'.format("================== a block of source codes ====================="))
        # print('{:>157}'.format(src))    
        lst = src.split('\n')
        for l in lst: 
            print('{:>157}'.format(l))
            
    else:
        print(src + "<===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle a block of code
        if "\n" in c: 
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c: 
            
            # if the code in string is like 'for k, v in abc:'
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env))
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

In [ ]:
blocks = """
if to is None: 
    to_f,from_f = f.__base__.__init__,f.__init__
else:          
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        """
lst = blocks.split('\n')
for l in lst: 
    print('{:>157}'.format(l))

                                                                                                                                                             
                                                                                                                                              if to is None: 
                                                                                                                 to_f,from_f = f.__base__.__init__,f.__init__
                                                                                                                                              else:          
                                                                                                   to_f,from_f = to.__init__ if isinstance(to,type) else to,f
                                                                                                                                                             


### add feature: if `abc` from `for k, v in abc:` is empty, then skip the current iteration

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('{:>157}'.format("===inside a block==="))
        print('{:>157}'.format(src))
        print(src + "<===== source code =======") 

    else:
        print('{:>157}'.format("==================================================="))

    
        # print out markers for a block of source and a line source code 
        if "\n" in src: 
            lst = src.split('\n')
            for l in lst: 
                print('{:>157}'.format(l))
            print(src + "<===== source code =======") 
            print('{:^157}'.format("<===== source code ======="))

        else:
            print('{:>157}'.format(src))        
            print(src + "<===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        
        # handle a block of code
        if "\n" in c: 
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if bool(abc): 
                print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            print(f"{c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

In [ ]:
dict1 = {'a':1}
lst1 = [1]
bool(dict1)

True

In [ ]:
dict1 = {}
lst1 = []
bool(dict1)

False

### nicer output spacing

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        print(src + "<===== source code =======") 

    else:
        print('\n')
        print('{:>157}'.format("==================================================="))

    
        # print out markers for a block of source and a line source code 
        if "\n" in src: 
            lst = src.split('\n')
            for l in lst: 
                print('{:>157}'.format(l))
            print('\n')

            print('{:^157}'.format("<===== source code ======="))
            print(src) 
            print('\n')

        else:
            print('{:>157}'.format(src))        
            print(src + "<===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            print(f"{c} => ")
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            print(f"{c} => {variable}: {eval(variable)}")
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                print(f"{c} => {cond}: {eval(cond)}")      
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    print(f"{c} => {printc} : ")
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    print(f"{c} => {cond}: {eval(cond)}")
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if bool(abc): 
                print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    print(f"{c} => {printmsg} : ")          
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                print(f"{c} => {printc} : ")
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### print output left and right reverted

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k, v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:<157}'.format("============================= source code ================================"))
        print('{:<157}'.format(src))        
    
        # print out markers for a block of source and a line source code 
        # if "\n" in src: 
        #     lst = src.split('\n')
        #     for l in lst: 
        #         print('{:>157}'.format(l))
        #     print('\n')

            # print('{:^157}'.format("<===== source code ======="))
            # print(src)  # print the block of src for the second time but on the left hand side
            # print('\n')

        # else:
        #     print('{:>157}'.format(src))        
            # print(src + "<===== source code =======") 
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            # print(f"{c} => {variable}: {eval(variable)}")
            output = f"{c} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if bool(abc): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (', ') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(', ')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                # print(f"{c} => {printc} : ")
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            output = f"{c} => {c} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### add feature: print out the entire source code and use ====== to mark the source code line at debugging

In [ ]:
import inspect
import fastcore.meta as fm
defaults.src = inspect.getsource(fm.delegates)

In [ ]:
defaults.src

'def delegates(to:FunctionType=None, # Delegatee\n              keep=False, # Keep `kwargs` in decorated function?\n              but:list=None): # Exclude these parameters from signature\n    "Decorator: replace `**kwargs` in signature with params from `to`"\n    if but is None: but = []\n    def _f(f):\n        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\n        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f\n        from_f = getattr(from_f,\'__func__\',from_f)\n        to_f = getattr(to_f,\'__func__\',to_f)\n        if hasattr(from_f,\'__delwrap__\'): return f\n        sig = inspect.signature(from_f)\n        sigd = dict(sig.parameters)\n        k = sigd.pop(\'kwargs\')\n        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()\n              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}\n        anno = {k:v for k,v in to_f.__annotations__.items() if k

In [ ]:
lst = defaults.src.split('\n')
l1 = lst[6]
l1

'        if to is None: to_f,from_f = f.__base__.__init__,f.__init__'

In [ ]:
l1.strip()
l1

'        if to is None: to_f,from_f = f.__base__.__init__,f.__init__'

In [ ]:
block1 = """
if to is None: to_f,from_f = f.__base__.__init__,f.__init__
else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        """
block1

'\nif to is None: to_f,from_f = f.__base__.__init__,f.__init__\nelse:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f\n        '

In [ ]:
lst[6].strip() in block1

True

In [ ]:
lst[-1], lst[-4]

('',
 "        if hasattr(from_f, '__annotations__'): from_f.__annotations__.update(anno)")

In [ ]:
bool(lst[-2])

True

### defaults.block is no longer needed I think

In [ ]:
class dbcolors:
    g = '\033[92m' #GREEN
    y = '\033[93m' #YELLOW
    r = '\033[91m' #RED
    reset = '\033[0m' #RESET COLOR

In [ ]:
def colorize(): pass

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:<157}'.format("============================= source code ================================"))
        print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
            else: 
                print('{:<157}'.format(l))
        print('\n')
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            # print(f"{c} => {variable}: {eval(variable)}")
            output = f"{c} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                # print(f"{c} => {printc} : ")
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            output = f"{c} => {c} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### add features to colorize question, feature and comment

In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:<157}'.format("============================= source code ================================"))
        print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
            else: 
                print('{:<157}'.format(l))
        # print out the example
        print('{:<157}'.format(defaults.eg))
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        c1 = None # use it as container for colored string
        # handle comment
        if "#comment#" in c:
            _,c1 = colorize(c)
            # print('{:>157}'.format(c1))   
            print(c1)
            continue
        else:

            # c is normal string, c1 is colored string, both are without #question# or #feature# or #comment#
            c,c1 = colorize(c)
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            exec(c, globals().update(env)) 
            variable = c.partition(" = ")[0]
            output = f"{c1} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        # handle comment
        # elif "#comment#" in c:
        #     # c,c1 = colorize(c)
        #     print('{:>157}'.format(c1))   
        #     print(c1)
        
        # handle evaluation
        else: 
            # c1 is to color #feature# or #question#
            # c,c1 = colorize(c)
            output = f"{c} => {c1} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time 

In [ ]:
blocks = """
if to is None: 
    to_f,from_f = f.__base__.__init__,f.__init__
else:          
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        """
lst = blocks.split('\n')
for l in lst: 
    print('{:>157}'.format(l))

                                                                                                                                                             
                                                                                                                                              if to is None: 
                                                                                                                 to_f,from_f = f.__base__.__init__,f.__init__
                                                                                                                                              else:          
                                                                                                   to_f,from_f = to.__init__ if isinstance(to,type) else to,f
                                                                                                                                                             


## Alignright a block of codes

In [ ]:
blocks = """
if to is None:
    to_f,from_f = f.__base__.__init__,f.__init__
else:
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        """

In [ ]:
#|export
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(l) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [ ]:
alignright("this is me")
alignright('\x1b[91mthis is me\x1b[0m') # can we align this colored line all the way to the right?

                                                                                                                                                   this is me
                                                                                                                                          this is me


In [ ]:
def dbprint(src:str, # the source to debug in str
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            cmt:str=None, # add colorful comment
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:<157}'.format("============================= source code ================================"))
        print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
            else: 
                print('{:<157}'.format(l))
        # print out the example
        print('{:<157}'.format(defaults.eg))
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
#         c1 = None # use it as container for colored string
#         # handle comment
#         if "#comment#" in c:
#             _,c1 = colorize(c)
#             # print('{:>157}'.format(c1))   
#             print(c1)
#             continue
#         else:

#             # c is normal string, c1 is colored string, both are without #question# or #feature# or #comment#
#             c,c1 = colorize(c)
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            exec(c, globals().update(env)) 
            variable = c.partition(" = ")[0]
            output = f"{c1} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        # handle comment
        # elif "#comment#" in c:
        #     # c,c1 = colorize(c)
        #     print('{:>157}'.format(c1))   
        #     print(c1)
        
        # handle evaluation
        else: 
            # c1 is to color #feature# or #question#
            # c,c1 = colorize(c)
            output = f"{c} => {c1} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time 

In [ ]:
clst = "this is me".split('\n') 
len(clst)

1

In [ ]:
#|export
class dbcolors:
    g = '\033[92m' #GREEN
    y = '\033[93m' #YELLOW
    r = '\033[91m' #RED
    reset = '\033[0m' #RESET COLOR

In [ ]:
#|export
def colorize(cmt, color:str=None):
    if color == "g":
        return dbcolors.g + cmt + dbcolors.reset
    elif color == "y":
        return dbcolors.y + cmt + dbcolors.reset
    elif color == "r":
        return dbcolors.r + cmt + dbcolors.reset
    else: 
        return cmt

In [ ]:
print(colorize("this is me", "r") + colorize("this is me", "y"))

this is methis is me


In [ ]:

def dbprint(src:str, # the source to debug in str
            cmt:str,
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # whether dbprint is working inside a for in loop or any kind of a loop
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:<157}'.format("============================= source code ================================"))
        print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')

        ccount = 0
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
                
                if bool(cmt):
                    clst = cmt.split('\n') 
                    if ccount <= len(clst)-1:
                        print('{:>157}'.format(colorize(clst[ccount])))
                        ccount = ccount + 1

            else: 
                print('{:<157}'.format(l))

        # print out the example
        print('{:<157}'.format(defaults.eg))
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment
        elif " = " in c: # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            # print(f"{c} => {variable}: {eval(variable)}")
            output = f"{c} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                # print(f"{c} => {printc} : ")
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            output = f"{c} => {c} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### add feature when handling assignment: 2. when = occur before if; 1. when no if only =

In [ ]:

def dbprint(src:str, # the source to debug in str
            cmt:str,
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # Inside the source code, ff you ever add a block of multiline codes like `for in` or `if`, and run dbprint for each line of the block, then set 
    # defaults.block to True
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:#^157}'.format(" source code with lines under investigation "))
        print('\n')
        # print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')

        ccount = 0
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
                
                if bool(cmt):
                    clst = cmt.split('\n') 
                    if ccount <= len(clst)-1:
                        print('{:>157}'.format(colorize(clst[ccount], "r")))
                        ccount = ccount + 1

            else: 
                print('{:<157}'.format(l))

        # print out the example
        print('{:<157}'.format(defaults.eg))
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment: 2. when = occur before if; 1. when no if only =
        elif ("=" in c and "if" not in c) or ("=" in c and c.find("=") < c.find("if")): # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            # print(f"{c} => {variable}: {eval(variable)}")
            output = f"{c} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                # print(f"{c} => {printc} : ")
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            output = f"{c} => {c} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

### alignright the colored comment

In [ ]:
#|export
def dbprint(src:str, # the source to debug in str
            cmt:str,
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert and run your codes and give readable output during debugging. Caution 1: \
    avoid using the same variable name used in both global and local scopes, e.g., \
    use `k` in the func and use `k` again inside a for loop inside the func.\
    Caution 2: make sure to include all the necessary env variables to avoid \
    the same variable with different values from different scopes. Caution 3: when an env variable is updated, \
    then you need to includ it again in the next dbprint. Caution 4: be strict on the spaces, e.g., `for k,v in` \
    and `a = createsth(...)`"
    
    # Inside the source code, ff you ever add a block of multiline codes like `for in` or `if`, and run dbprint for each line of the block, then set 
    # defaults.block to True
    if defaults.block == True:
        print('\n')
        print('{:>157}'.format("===source inside a block==="))
        print('{:>157}'.format(src))
        print('\n')
        # print(src + "<===== source code =======") 

    else:
        print('\n')
        # print('{:>157}'.format("======================== source code ==========================="))
        print('{:#^157}'.format(" source code with lines under investigation "))
        print('\n')
        # print('{:<157}'.format(src))        
        
        # print the source code of the function
        lst = defaults.src.split('\n')

        ccount = 0
        for l in lst: 
            if bool(l) and l.strip() in src: # how to make sure all these ls are close to each other???
                print('{:=<157}'.format(l))
                ccount = ccount + 1
                
                if bool(cmt): # make sure the comments are colored and aligned to the most right
                    # if this is the last srcline of the srcblock under investigation
                    numsrclines = len(src.split("\n"))
                    if ccount == numsrclines:
                        colcmt = colorize(cmt, "r")
                        alignright(colcmt)
                    # clst = cmt.split('\n') 
                    # if ccount <= len(clst)-1:
                    #     # print('{:>157}'.format(colorize(clst[ccount], "r")))
                    #     colcmt = colorize(clst[ccount], "r")
                    #     alignright(colcmt)
                    #     ccount = ccount + 1

            else: 
                print('{:<157}'.format(l))

        # print out the example
        print('{:<157}'.format(defaults.eg))
        
    
    # trial and error version for real code, still not quite why globals vs locals work in exec and eval
    for c in code:
        print("\n")
        
        # handle a block of code
        if "\n" in c: 
            output = f"Running your code block => "
            print('{:<157}'.format(c))       
            print('{:>157}'.format(output))  
            print('The code block printout => : ')
            block = ast.parse(c, mode='exec')
            exec(compile(block, '<string>', mode='exec'), globals().update(env))
        
        # handle assignment: 2. when = occur before if; 1. when no if only =
        elif ("=" in c and "if" not in c) or ("=" in c and c.find("=") < c.find("if")): # make sure assignment and !== and == are differentiated
            
            # print('k' in locals())
            exec(c, globals().update(env)) 
            # print('k' in locals())
            variable = c.partition(" = ")[0]
            # print(f"{c} => {variable}: {eval(variable)}")
            output = f"{c} => {variable}: {eval(variable)}"
            print('{:>157}'.format(output))       
            
        # handle if statement
        # Note: do insert code like this : `if abc == def: print(abc)`, print is a must
        elif "if" in c: 
            cond = re.search('if (.*?):', c).group(1)
            
            # when code in string is like 'if abc == def:'
            if c.endswith(':'):
                
                # print ... 
                # print(f"{c} => {cond}: {eval(cond)}")      
                output = f"{c} => {cond}: {eval(cond)}"
                print('{:>157}'.format(output))
                
            # when code in string is like 'if abc == def: print(...)'
            else: 
                # if the cond is true, then print ...
                if eval(cond):
                    
                    # "if abc == def: print(abc)".split(': ', 2)[1] to get 'print(abc)'
                    printc = c.split(': ', 1)[1]
                    # print(f"{c} => {printc} : ")
                    output = f"{c} => {printc} : "
                    print('{:>157}'.format(output))      
                    exec(c, globals().update(env))
                    
                # if cond is false, then print ...
                else: 
                    # print(f"{c} => {cond}: {eval(cond)}")
                    output = f"{c} => {cond}: {eval(cond)}"
                    print('{:>157}'.format(output))   
                
                
        # handle for in statement
        elif "for " in c and " in " in c:           
            
            # in example like 'for k, v in abc:' or `for k, v in abc: print(...)`, if abc is empty
            # step 1: access abc
            # get the substring between 'in ' and ':', which is like 'abc'
            abc = re.search('in (.*?):', c).group(1)
            # if abc is empty dict or list: print and pass
            if not bool(eval(abc)): 
                # print(f'{c} => {abc} is an emtpy {type(eval(abc))}')
                output = f'{c} => {abc} is an emtpy {type(eval(abc))}'
                print('{:>157}'.format(output))   
                continue 
                # The break statement can be used if you need to break out of a for or while loop and move onto the next section of code.
                # The continue statement can be used if you need to skip the current iteration of a for or while loop and move onto the next iteration.
            
            # if the code in string is like 'for k, v in abc:', there is no more code after `:`
            if c.endswith(':'):
                
                # get the substring between 'for ' and ' in', which is like 'k, v'
                variables = re.search('for (.*?) in', c).group(1)
                
                # if variables has a substring like ', ' inside
                if (',') in variables: 
                    
                    # split it by ', ' into a list of substrings
                    vl = variables.split(',')
                    key = vl[0]
                    value = vl[1]
                    
                    # make sure key and value will get evaluated first before exec run
                    # printc is for exec to run
                    printc = "print(f'{key}:{eval(key)}, {type(eval(key))} ; {value}:{eval(value)}, {type(eval(value))}')" 
                    # printmsg is for reader to understand with ease
                    printmsg = "print(f'key: {key}, {type(key)} ; value: {value}, {type(value)}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")      
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                
                else:
                    printc = "print(f'{variables} : {eval(variables)}')"
                    printmsg = "print(f'i : {variables}')"
                    c1 = c + " " + printc
                    # print(f"{c} => {printmsg} : ")     
                    output = f"{c} => {printmsg} : "
                    print('{:>157}'.format(output))   
                    exec(c1, globals().update(env))
                    
            # if the code in string is like 'for k, v in abc: print(abc)'
            else:                 
                # "for k, v in abc: print(k)".split(': ', 1)[1] to get 'print(k)'
                printc = c.split(': ', 1)[1]
                # print(f"{c} => {printc} : ")
                output = f"{c} => {printc} : "
                print('{:>157}'.format(output))   
                exec(c, globals().update(env)) # we can't use eval to run `for in` loop, but exec can.
            ### Note: we shall not use the expression like `for k, v in abc print(abc)`
            ### Note: we shall not use the expression like `for k, v in abc if k == def`
        
        
        # handle evaluation
        else: 
            # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
            output = f"{c} => {c} : {eval(c, globals().update(env))}"
            print('{:>157}'.format(output))   
            
        # the benefit of using global().update(env) is 
        # to ensure we don't need to include the same env for the second time

## insert dbprint and make a new source function for debugging line by line

In [ ]:
def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    srcode = inspect.getsource(eval(name, globals().update(env)))
    lstxt = srcode.split(srcline)
    retn = "\n        return None\n"
    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
    for l in src2print.split("\n"):  # print out the debuggable version of delegates
        print(l)
    src2db = lstxt[0] + dbcode + srcline + retn + lstxt[1]
    exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
    globals().update({name:eval(name)})
    return eval(name) # give this debuggable version of delegates to the notebook context

In [ ]:
insert2debug??

Signature: insert2debug(name: str, srcline: str, dbcode: str, **env)
Source:   
def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    srcode = inspect.getsource(eval(name, globals().update(env)))
    lstxt = srcode.split(srcline)
    retn = "\n        return None\n"
    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
    for l in src2print.split("\n"):  # print out the debuggable version of delegates
        print(l)
    src2db = lstxt[0] + dbcode + srcline + retn + lstxt[1]
    exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
    globals().upd

###  remove `return None`, so that we can run multiple `delegates` in one cell without error

In [ ]:
def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    srcode = inspect.getsource(eval(name, globals().update(env)))
    lstxt = srcode.split(srcline)
    # retn = "\n        return None\n"
    retn = ""
    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
    for l in src2print.split("\n"):  # print out the debuggable version of delegates
        print(l)
    src2db = lstxt[0] + dbcode + srcline + retn + lstxt[1]
    exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
    # globals().update({name:eval(name)})
    return eval(name) # give this debuggable version of delegates to the notebook context

In [ ]:

def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
    else: 
        # srcode = inspect.getsource(eval(name, globals().update(env)))
        srcode = defaults.src
    lstxt = srcode.split(srcline)
    retn = "" # retn = "\n        return None\n" to exit the function, "" to continue on
    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
    for l in src2print.split("\n"):  # print out the debuggable version of delegates
        print(l)
    src2db = lstxt[0] + dbcode + srcline + retn + lstxt[1]
    if defaults.multi: defaults.deb = src2db
    exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
    # globals().update({name:eval(name)})
    return eval(name) # give this debuggable version of delegates to the notebook context

### major problems solved

In [ ]:

def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 run:bool=True, # run exec or not
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
    else: 
        # srcode = inspect.getsource(eval(name, globals().update(env)))
        srcode = defaults.src
    lstxt = srcode.split(srcline)
    retn = "" # retn = "\n        return None\n" to exit the function, "" to continue on
    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
            
    src2db = lstxt[0] + dbcode + srcline + retn + lstxt[1]
    # if defaults.multi: defaults.deb = src2db
    defaults.deb = src2db
    
    # to exec 
    if run: 
        exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
        # globals().update({name:eval(name)})
        return eval(name) # give this debuggable version of delegates to the notebook context
    # to not exec but only get the full debuggable source 
    else: 
        return None

### debugging error

In [ ]:

def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 run:bool=True, # run exec or not
                 dberror:bool=False, # choose to debug error in source code by setting return None or not
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
    else: 
        # srcode = inspect.getsource(eval(name, globals().update(env)))
        srcode = defaults.src
    lstxt = srcode.split(srcline)
    retn = "" 
    if dberror: 
        retn = retn + "\n        return None\n" #  to exit the function, "" to continue on


    insert = colorize(dbcode, "g") + colorize(srcline, "r") + colorize(retn, "y")
    src2print = lstxt[0] + insert + lstxt[1]
            
    src2db = lstxt[0] + dbcode + retn + srcline + lstxt[1] # make sure return is before srcline and after dbcode
    # if defaults.multi: defaults.deb = src2db
    defaults.deb = src2db
    
    # to exec 
    if run: 
        exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
        # globals().update({name:eval(name)})
        return eval(name) # give this debuggable version of delegates to the notebook context
    # to not exec but only get the full debuggable source 
    else: 
        return None

### to export src2print for coloring srcline, dbcode and return

In [ ]:

def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 run:bool=True, # run exec or not
                 dberror:bool=False, # choose to debug error in source code by setting return None or not
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
    else: 
        # srcode = inspect.getsource(eval(name, globals().update(env)))
        srcode = defaults.src
    lstxt = srcode.split(srcline)
    
    
    retn = "" 
    if dberror: 
        retn = retn + "\n        return None\n" #  to exit the function, "" to continue on


    insert = colorize(dbcode, "g") + colorize(retn, "y") + colorize(srcline, "r") 
    src2print = lstxt[0] + insert + lstxt[1]
            
    src2db = lstxt[0] + dbcode + retn + srcline + lstxt[1] # make sure return is before srcline and after dbcode
    # save the entire source code with dbprints
    defaults.deb = src2db
    # save the entire source code with dbprints for color printing
    defaults.debp = src2print
    
    # to exec 
    if run: 
        exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
        # globals().update({name:eval(name)})
        return eval(name) # give this debuggable version of delegates to the notebook context
    # to not exec but only get the full debuggable source 
    else: 
        return None

### To make all selected dbcodes colorful

In [ ]:
#|export
def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 run:bool=True, # run exec or not
                 dberror:bool=False, # choose to debug error in source code by setting return None or not
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
        srcodep = defaults.debp
        lstxtp = srcodep.split(srcline.strip()) # make sure the split is done properly
    else: 
        # srcode = inspect.getsource(eval(name, globals().update(env)))
        srcode = defaults.src
        
    lstxt = srcode.split(srcline)
    
    
    retn = "" 
    if dberror: 
        retn = retn + "\n        return None\n" #  to exit the function, "" to continue on

    
    insert = colorize(dbcode, "g") + colorize(retn, "y") + colorize(srcline, "r") 
    if bool(defaults.deb):
        src2print = lstxtp[0] + insert + lstxtp[1]
    else:
        src2print = lstxt[0] + insert + lstxt[1]
            
    src2db = lstxt[0] + dbcode + retn + srcline + lstxt[1] # make sure return is before srcline and after dbcode
    # save the entire source code with dbprints
    defaults.deb = src2db
    # save the entire source code with dbprints for color printing
    defaults.debp = src2print
    
    # to exec 
    if run: 
        exec(src2db, globals().update({'srcline': srcline})) # now a debuggable version of delegates is available to use
        # globals().update({name:eval(name)})
        return eval(name) # give this debuggable version of delegates to the notebook context
    # to not exec but only get the full debuggable source 
    else: 
        return None

In [ ]:
print(colorize("", "y") + colorize("1", "r"))

1


In [ ]:
insert2debug??

Signature:
insert2debug(
    name: str,
    srcline: str,
    dbcode: str,
    run: bool = True,
    dberror: bool = False,
    **env,
)
Source:   
def insert2debug(name:str, # name of a function to debug, e.g., delegates
                 srcline:str, # e.g., "        if hasattr(from_f,'__delwrap__'): return f"
                 dbcode:str,  # str, e.g., "dbprint(...)"
                 run:bool=True, # run exec or not
                 dberror:bool=False, # choose to debug error in source code by setting return None or not
                 **env):
    "select a line or a block of source code and insert a dbprint above it and only output this dbprint result."
    
    # defaults.multi default to False, unless set True, defaults.deb is default to None before debugging srcode
    if defaults.multi and bool(defaults.deb): 
        srcode = defaults.deb
        srcodep = defaults.debp
        lstxtp = srcodep.split(srcline.strip()) # make sure the split is done properly
    else: 
        # s

In [ ]:
#|export
from fastcore.foundation import L

In [ ]:

def dbsrclines(srcname:str, # name of the source code, e.g., delegates
               lines:list = None # if None then print all e.g., defaults.src2dbp.delegates
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if not bool(lines):
        for i in srcdblist:
            pprint(i[0][0], width=157)
            pprint(i[0][1], width=157)
            print("")
        return None
    
    if len(lines) > 1: 
        defaults.multi = True
        for i in eval("srcdblist" + str(lines)):
            delegates = insert2debug(srcname, i[0][0], i[0][1])
    else: 
        item = eval("srcdblist" + str(lines))
        delegates = insert2debug(srcname, item[0][0], item[0][1])
    

    defaults.multi = False
    defaults.deb = None
    return delegates

In [ ]:
for i in L([[(1,2)],[(2,2)]]): 
    print(i)

[(1, 2)]
[(2, 2)]


## Exporting the debuggable source and major problems solved

In [ ]:

def dbsrclines(srcname:str, # name of the source code, e.g., delegates
               lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False # get the full debuggable source code
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if not bool(lines) and dbsrc == False: # to print out the source code and mark all the srclines 
        for i in srcdblist:
            pprint(i[0][0], width=157)
            pprint(i[0][1], width=157)
            print("")
        return None
     
    if dbsrc and not bool(lines): # to print out the entire debuggable source code
        defaults.multi = True
        for i in srcdblist:
            insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec just add up debuggable source
        # export the debuggable source
        defaults.src2dbp.delegatesdb = defaults.deb
        pprint(defaults.src2dbp.delegatesdb, width=157)
        defaults.deb = None
        defaults.multi = False
        return None
    
    if len(lines) > 1: 
        defaults.multi = True
        for i in eval("srcdblist" + str(lines)):
            delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        delegates = insert2debug(srcname, item[0][0], item[0][1])

    pprint(defaults.deb, width=157) # print the debuggable source

    defaults.multi = False
    defaults.deb = None
    return delegates

### improve the print feature of dbscrlines 

In [ ]:

def dbsrclines(srcname:str, # name of the source code, e.g., delegates
               lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False # get the full debuggable source code
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if not bool(lines) and dbsrc == False: # to print out the source code and mark all the srclines 
        # put all srclines into a single string
        srclines = ""
        for i in srcdblist:
            srclines = srclines + i[0][0]
        
        for l in defaults.src.split("\n"):
            if l in srclines:
                print('{:=<157}'.format(l))
            else: 
                print('{:<157}'.format(l))
        print("")
        return None
     
    if dbsrc and not bool(lines): # set dbsrc to true to export the entire debuggable source code to defaults.src2dbp.delegatesdb
        defaults.multi = True
        for i in srcdblist:
            insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec just add up debuggable source
        # export the debuggable source
        defaults.src2dbp.delegatesdb = defaults.deb
        # pprint(defaults.src2dbp.delegatesdb, width=157)
        defaults.deb = None
        defaults.multi = False
        return None
    
    if len(lines) > 1: 
        defaults.multi = True
        for i in eval("srcdblist" + str(lines)):
            delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        delegates = insert2debug(srcname, item[0][0], item[0][1])

    pprint(defaults.deb, width=157) # print the debuggable source

    defaults.multi = False
    defaults.deb = None
    return delegates

### add return to the last dbcode for debugging examples which cause errors

In [ ]:

def dbsrclines(srcname:str, # name of the source code, e.g., delegates
               lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False, # get the full debuggable source code
               retn:bool = False # choose to add return None after the last dbcode
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if not bool(lines) and dbsrc == False: # to print out the source code and mark all the srclines 
        # put all srclines into a single string
        srclines = ""
        for i in srcdblist:
            srclines = srclines + i[0][0]
        
        for l in defaults.src.split("\n"):
            if l in srclines:
                print('{:=<157}'.format(l))
            else: 
                print('{:<157}'.format(l))
        print("")
        return None
     
    if dbsrc and not bool(lines): # set dbsrc to true to export the entire debuggable source code to defaults.src2dbp.delegatesdb
        defaults.multi = True
        for i in srcdblist:
            insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec just add up debuggable source
        # export the debuggable source
        defaults.src2dbp.delegatesdb = defaults.deb
        # pprint(defaults.src2dbp.delegatesdb, width=157)
        defaults.deb = None
        defaults.multi = False
        return None
    
    if len(lines) > 1: 
        defaults.multi = True
        lst = eval("srcdblist" + str(lines))
        for idx, i in zip(range(len(lst)), lst): # add retn to the last dbcode
            if retn and idx == len(lst)-1:
                delegates = insert2debug(srcname, i[0][0], i[0][1], dberror=True) ### add dberror to insert2debug
            else:
                delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        delegates = insert2debug(srcname, item[0][0], item[0][1])

    pprint(defaults.deb, width=157) # print the debuggable source

    defaults.multi = False
    defaults.deb = None
    return delegates

In [ ]:

def dbsrclines(lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False, # get the full debuggable source code
               retn:bool = False # choose to add return None after the last dbcode
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
#     if not bool(lines) and dbsrc == False: # to print out the source code and mark all the srclines 
#         # put all srclines into a single string
#         srclines = ""
#         for i in srcdblist:
#             srclines = srclines + i[0][0]
        
#         for l in defaults.src.split("\n"):
#             if l in srclines:
#                 print('{:=<157}'.format(l))
#             else: 
#                 print('{:<157}'.format(l))
#         print("")
#         return None
     
    if dbsrc and not bool(lines): # set dbsrc to true to export the entire debuggable source code to defaults.src2dbp.delegatesdb
        defaults.multi = True
        for i in srcdblist:
            insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec just add up debuggable source
        # export the debuggable source
        defaults.src2dbp.delegatesdb = defaults.deb
        # pprint(defaults.src2dbp.delegatesdb, width=157)
        defaults.deb = None
        defaults.multi = False
        return None
    
    if len(lines) > 1: 
        defaults.multi = True
        lst = eval("srcdblist" + str(lines))
        for idx, i in zip(range(len(lst)), lst): # add retn to the last dbcode
            if retn and idx == len(lst)-1:
                delegates = insert2debug(srcname, i[0][0], i[0][1], dberror=True) ### add dberror to insert2debug
            else:
                delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        if retn:
            delegates = insert2debug(srcname, item[0][0], item[0][1], dberror=True)  ### add dberror to insert2debug           
        else:
            delegates = insert2debug(srcname, item[0][0], item[0][1])

    pprint(defaults.deb, width=157) # print the debuggable source
    pprint(defaults.debp, width=157) # print the debuggable source
    for l in defaults.debp.split("\n"):
        print(l)

    defaults.multi = False
    defaults.deb = None
    return delegates

In [ ]:

def dbsrclines(lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False, # get the full debuggable source code
               retn:bool = False # choose to add return None after the last dbcode
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if len(lines) > 1: 
        defaults.multi = True
        lst = eval("srcdblist" + str(lines))
        for idx, i in zip(range(len(lst)), lst): # add retn to the last dbcode
            if retn and idx == len(lst)-1:
                delegates = insert2debug(srcname, i[0][0], i[0][1], dberror=True) ### add dberror to insert2debug
            else:
                delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        if retn:
            delegates = insert2debug(srcname, item[0][0], item[0][1], dberror=True)  ### add dberror to insert2debug           
        else:
            delegates = insert2debug(srcname, item[0][0], item[0][1])

    # print out the colored dbsrc selected
    for l in defaults.debp.split("\n"):
        print(l)

    defaults.multi = False
    defaults.deb = None
    return delegates

### add displaysavesrc into dbsrclines

In [ ]:
#|export
def dbsrclines(lines:list = None, # if None then print all e.g., defaults.src2dbp.delegates
               dbsrc:bool = False, # get the full debuggable source code
               retn:bool = False # choose to add return None after the last dbcode
              ): 
    "Doing one line or multilines of insert2debug on source code with dbprints."
    
    # display the entire debuggable source code, and save defaults.src2dbp.{srcname}db and defaults.src2dbp.{srcname} in pickle files.
    displaysavedbsrc(display=False)
    
    
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    if len(lines) > 1: 
        defaults.multi = True
        lst = eval("srcdblist" + str(lines))
        for idx, i in zip(range(len(lst)), lst): # add retn to the last dbcode
            if retn and idx == len(lst)-1:
                delegates = insert2debug(srcname, i[0][0], i[0][1], dberror=True) ### add dberror to insert2debug
            else:
                delegates = insert2debug(srcname, i[0][0], i[0][1])

    else: 
        item = eval("srcdblist" + str(lines))
        if retn:
            delegates = insert2debug(srcname, item[0][0], item[0][1], dberror=True)  ### add dberror to insert2debug           
        else:
            delegates = insert2debug(srcname, item[0][0], item[0][1])

    # print out the colored dbsrc selected
    for l in defaults.debp.split("\n"):
        print(l)

    defaults.multi = False
    defaults.deb = None
    return delegates

## Display the entire debuggable source with dbprints and save it to defaults.src2dbp....db

In [ ]:
b = "pple"
exec(f'a{b} = 3')

In [ ]:
apple

3

In [ ]:

def displaysavedbsrc():
    "save the entire debuggable source code in defaults.src2dbp.{srcname}db and display it with color"
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
     
    
    defaults.multi = True
    for i in srcdblist:
        insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec, just add up debuggable source
        
    # export the debuggable source
    # defaults.src2dbp.delegatesdb = defaults.deb
    exec(f'defaults.src2dbp.{srcname}db = defaults.deb')
    # pprint(defaults.src2dbp.delegatesdb, width=157)
    dbsrclines = ""
    for l in srcdblist:
        dbsrclines = dbsrclines + l[0][0]
    for l in defaults.deb.split('\n'):
        if l.strip() in dbsrclines:
            print(colorize(l, 'g'))
        else:
            print(l)
    defaults.deb = None
    defaults.multi = False
    return None

In [ ]:

def displaysavedbsrc():
    "display the entire debuggable source code, and save defaults.src2dbp.{srcname}db and defaults.src2dbp.{srcname} in pickle files."
    
    matchsrcorder() # "Match srcdbps list in the same order as the official source code."
    
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
     
    
    defaults.multi = True
    for i in srcdblist:
        insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec, just add up debuggable source
        
    # export the debuggable source
    # defaults.src2dbp.delegatesdb = defaults.deb
    exec(f'defaults.src2dbp.{srcname}db = defaults.deb')
    # pprint(defaults.src2dbp.delegatesdb, width=157)
    dbsrclines = ""
    for l in srcdblist:
        dbsrclines = dbsrclines + l[0][0]
    for l in defaults.deb.split('\n'):
        if l.strip() in dbsrclines:
            print(colorize(l, 'g'))
        else:
            print(l)
    defaults.deb = None
    defaults.multi = False
    
    # save defaults.src2dbp.{defaults.name} and defaults.src2dbp.{defaults.name}db into pickle file
    _save_dbsrcstr_dbcodelist()
    
    return None

In [ ]:
#|export
def displaysavedbsrc(display:bool=True):
    "display the entire debuggable source code, and save defaults.src2dbp.{srcname}db and defaults.src2dbp.{srcname} in pickle files."
    
    matchsrcorder() # "Match srcdbps list in the same order as the official source code."
    
    srcname = defaults.name
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
     
    
    defaults.multi = True
    for i in srcdblist:
        insert2debug(srcname, i[0][0], i[0][1], run=False) # don't exec, just add up debuggable source
        
    # export the debuggable source
    # defaults.src2dbp.delegatesdb = defaults.deb
    exec(f'defaults.src2dbp.{srcname}db = defaults.deb')
    
    if display:
        dbsrclines = ""
        for l in srcdblist:
            dbsrclines = dbsrclines + l[0][0]
        for l in defaults.deb.split('\n'):
            if l.strip() in dbsrclines:
                print(colorize(l, 'g'))
            else:
                print(l)

    defaults.deb = None
    defaults.multi = False
    
    # save defaults.src2dbp.{defaults.name} and defaults.src2dbp.{defaults.name}db into pickle file
    _save_dbsrcstr_dbcodelist()
    
    return None

## save defaults.src2dbp.{defaults.name} and defaults.src2dbp.{defaults.name}db into pickle file

In [ ]:
#|export
def _save_dbsrcstr_dbcodelist():
    
    path = Path(f"db/{defaults.name}")

    if path.is_file():
        with open(f"db/{defaults.name}", "wb") as fp:   # Unpickling
            eval(f'pickle.dump(defaults.src2dbp.{defaults.name}, fp)')
    else:
        print(f"Warning: there is no such file named db/{defaults.name}.")
        pass

    path = Path(f"db/{defaults.name}db")
    if path.is_file():
        with open(f"db/{defaults.name}db", "wb") as fp:   # Unpickling
            eval(f'pickle.dump(defaults.src2dbp.{defaults.name}db, fp)')
    else:
        print(f"Warning: there is no such file named db/{defaults.name}db.")
        pass


## check official source against debuggable source

### Add markers and raise error when there is update of the srcode

In [ ]:

def checksrc(srcname):
    "check src code against dbsource. Also the latest srcode is stored inside defaults.src."
    defaults.src = inspect.getsource(eval(srcname))

    # file1 = open("db/" + srcname + ".txt","r+") 

    # now dbsrc == defaults.src2dbp.delegatesdb
    dbsrc = eval("defaults.src2dbp." + srcname + "db")
    # dbsrc = file1.read()
    # file1.close()

    lst = defaults.src.split('\n')
    for l in lst: 

        if bool(dbsrc) and l.strip() in dbsrc:
            print('{:<157}'.format(l))
        else: 
            print('{:=<157}'.format(l))

In [ ]:
print(u'\u2713')

✓


In [ ]:
# #|export
# def checksrc():
#     "check src code against dbsource. Also the latest srcode is stored inside defaults.src."
#     srcname = defaults.name
#     defaults.src = inspect.getsource(eval(srcname))

#     # file1 = open("db/" + srcname + ".txt","r+") 

#     # now dbsrc == defaults.src2dbp.delegatesdb
#     dbsrc = eval("defaults.src2dbp." + srcname + "db")
#     # dbsrc = file1.read()
#     # file1.close()

#     lst = defaults.src.split('\n')
#     for l in lst: 
#         if bool(dbsrc) and l.strip() in dbsrc:
#             marker = f'(u\'\u2713\')'
#             indent = defaults.margin - len(l) - len(marker)
#             print(l + "="*indent + marker)    
#         else: 
#             print('{:=<157}'.format(l))
            
        
            
#     if bool(dbsrc) == False: 
#     print(f'your debuggable srcode is empty. You have not written any, or you have lost your defaults.src2dbp.{srcname}db content.')

In [ ]:

def checksrc():
    "check src code against dbsource. Also the latest srcode is stored inside defaults.src."
    srcname = defaults.name
    defaults.src = inspect.getsource(eval(srcname))

    # now dbsrc == defaults.src2dbp.delegatesdb
    dbsrc = eval("defaults.src2dbp." + srcname + "db")
    count = 0

    lst = defaults.src.split('\n')
    for l in lst: 
        if not bool(dbsrc):
            print(l)
        elif bool(dbsrc) and l.strip() in dbsrc:
            tick = f'(\u2713)'
            indent = defaults.margin - len(l) - len(tick)
            print(l + " "*indent + tick)    
        else: 
            print('{:=<157}'.format(l))
            count = count + 1
            
    if bool(dbsrc) == False: 
        print(f'your debuggable srcode is empty. You have not written any, or you have lost your defaults.src2dbp.{srcname}db or db/{srcname}db file.')
    if count > 0: 
        raise Exception(f'{srcname} has updated on {count} lines, you need to update your debuggable codes too.')

In [ ]:
#|export
from os.path import exists
import pickle

In [ ]:
# if exists("db/delegatesdb"):
#     # setup the folder and file when getting started. 
#     with open("db/delegatesdb", "rb") as fp:   
#       defaults.src2dbp.delegatesdb = pickle.load(fp)
# else:
#     pass

In [ ]:
#|export
def checksrc():
    "check src code against dbsource. Behind the scene, we are loading defaults.src2dbp.{srcname}db from pickle file \
    and the latest official srcode is stored inside defaults.src."
    srcname = defaults.name
    defaults.src = inspect.getsource(eval(srcname))

    # if exists(f"db/{srcname}db"):
    #     with open(f"db/{srcname}db", "rb") as fp:   
    #       exec(f"defaults.src2dbp.{srcname}db = pickle.load(fp)")
    # else:
    #     pass
    
    path = Path(f"db/{srcname}db")
    if path.is_file():
        with open(f"db/{srcname}db", "rb") as fp:   # Unpickling
            exec(f"defaults.src2dbp.{srcname}db = pickle.load(fp)")
    else:
        print(f"Warning: there is no such file named db/{srcname}db, so defaults.src2dbp.{srcname}db is empty.")
        pass
    
    # now dbsrc == defaults.src2dbp.delegatesdb
    dbsrc = eval("defaults.src2dbp." + srcname + "db")
    count = 0

    lst = defaults.src.split('\n')
    for l in lst: 
        if not bool(dbsrc):
            print(l)
        elif bool(dbsrc) and l.strip() in dbsrc:
            tick = f'(\u2713)'
            indent = defaults.margin - len(l) - len(tick)
            print(l + " "*indent + tick)    
        else: 
            print('{:=<157}'.format(l))
            count = count + 1
            
    if bool(dbsrc) == False: 
        print(f'your debuggable srcode is empty. You have not written any, or you have lost your defaults.src2dbp.{srcname}db or db/{srcname}db file.')
    if count > 0: 
        raise Exception(f'{srcname} has updated on {count} lines, you need to update your debuggable codes too.')

In [ ]:
#|export
def strip_ansi(source):
    return re.sub(r'\033\[(\d|;)+?m', '', source)

In [ ]:
#|export
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [ ]:
alignright("this is me")
alignright("\x1b[91mthis is me\x1b[0m")

                                                                                                                                                   this is me
                                                                                                                                                   this is me


## How to shuffle 

In [ ]:
src = "    def low(a, b=1):\n        c = b*2\n        return a + b + c"

In [ ]:
for l in src.split("\n"): print(l)

    def low(a, b=1):
        c = b*2
        return a + b + c


I have a list of tuples, and the first item of each tuple contains a line of the function above. The problem is the order of the lines are random.

In [ ]:
srcshuffled = [[("        return a + b + c", "this is the end of the func")],[("        c = b*2", "first line inside the func")],[("    def low(a, b=1):", "this is where signature is")]]

In [ ]:
for s in srcshuffled:
    print(s[0][0])

        return a + b + c
        c = b*2
    def low(a, b=1):


How can I reorder the list so that the first item of each tuple matches the function in the right order?

In [ ]:
srcorrect = [[("    def low(a, b=1):", "this is where signature is")],[("        c = b*2", "first line inside the func")], [("        return a + b + c", "this is the end of the func")]]

In [ ]:
for s in srcorrect:
    print(s[0][0])

    def low(a, b=1):
        c = b*2
        return a + b + c


my attempt

In [ ]:
for idx, l in zip(range(len(src.split("\n"))), src.split("\n")):
    print(idx)
    print(l)

0
    def low(a, b=1):
1
        c = b*2
2
        return a + b + c


In [ ]:
srcorrect1 = []
for l in src.split("\n"):
    for s in srcshuffled: 
        if l.strip() in s[0][0]:
            srcorrect1.append(s)         

In [ ]:
for s in srcorrect1:
    print(s[0][0])

    def low(a, b=1):
        c = b*2
        return a + b + c


In [ ]:

def matchsrcorder(srcdbps:list # the list contain all srclines and their dbcodes with random order
                 ):
    srcdbps1 = [] # a list to store the correct order of srclines and dbcodes
    for l in defaults.src.split("\n"):
        for idx, s in zip(range(len(srcdbps)), srcdbps):
            if l.strip() in s[0][0]:
                srcdbps.pop(idx)
                srcdbps1.append(s)  
    return srcdbps1

In [ ]:
# srcdbps = defaults.src2dbp.delegates # user input
srcdbps = []

In [ ]:
srcline = """
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
"""

In [ ]:
len(defaults.src.split(srcline)) # must be 2 to be right

2

In [ ]:
dbcode = """
        dbprint("        if keep: sigd['kwargs'] = k\\n        else: from_f.__delwrap__ = to_f", \
"what it does: you can keep **kwargs with keep=True, or you keep **kwargs out and add an attribute __delwrap__ with to_f.", \
"hasattr(from_f, '__delwrap__')", keep=keep, sigd=sigd, from_f=from_f, to_f=to_f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = """
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
"""

In [ ]:
len(defaults.src.split(srcline)) # must be 2 to be right

2

In [ ]:
dbcode = """
        dbprint("        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items() if v.default != inspect.Parameter.empty and k not in sigd and k not in but}", \
"what it does: f take the params from to and put them into a dict, k is name, v is the param in Parameter class, and make their kind KEYWORD_ONLY.", \
"if callable(to_f):\\n\
    if bool(inspect.signature(to_f).parameters.items()):\\n\
        for k,v in inspect.signature(to_f).parameters.items():\\n\
            print(f'k:v => {k}:{v}, type(v): {type(v)}')\\n\
    else:\\n\
        print(f'inspect.signature(to_f).parameters.items() is empty: {not bool(inspect.signature(to_f).parameters.items())}')\\n\
else:\\n\
    print(f'to_f is callable: {callable(to_f)}, so inspect.signature(to_f) will cause error')",\
to_f=to_f, k=k, but=but, sigd=sigd)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = "        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}"
dbcode = """
        dbprint("        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}", \
"what it does: check to_f's annotations (dict), and only select params with their annotations wanted by f or from_f.\\n\
what is __annotations__: a:int, b:int=1 are annotations\\n\
for k,v in to_f.__annotations__.items(): print(f'k:v => {k}:{v}')\\n\
    if the iterator above is empty, then print won't get executed.\\n\
    run this:\\n\
    for k, v in {}.items(): print(f'k:v => {k}:{v}') ", \
"hasattr(to_f, '__annotations__')", \
"if hasattr(to_f, '__annotations__'):\\n\
    if bool(to_f.__annotations__.items()) == False:\\n\
        print(f'to_f.__annotations__.items(): {to_f.__annotations__.items()}')\\n\
    else:\\n\
        for k, v in to_f.__annotations__.items():\\n\
            print(f'k:v => {k}:{v}')\\n\
else:\\n\
    print(f'We should expect error from the code of try-except.')\\n\
    try:\\n\
        to_f.__annotations__\\n\
    except AttributeError as e:\\n\
        print(e)", \
but=but, k=k, sigd=sigd, to_f=to_f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = """
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
"""

# watch out: to convert srcline into actual strings for the first arg in dbprint, use '\\n\' instead of just '\n'
dbcode = """
\n        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\\n\        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"UseCase1: when to is superclass to f\\nUseCase2: when to is a class but not superclass to f. UseCase3: other combinations",\
"if to is None:\\n\
    to_f,from_f = f.__base__.__init__,f.__init__\\n\
else:\\n\
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"to_f", "from_f", "hasattr(to_f, '__annotations__')", "hasattr(from_f, '__annotations__')", \
f=f, to=to)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = "        if hasattr(from_f,\'__delwrap__\'): return f"
dbcode = """
        dbprint("if hasattr(from_f,'__delwrap__'): return f", \
"If f or from_f has __delwrap__, it means it's happy with all params and give up on **kwargs. So no more params needed from other `to`s", \
"from_f", "f", "hasattr(from_f, '__delwrap__')", "if hasattr(from_f,'__delwrap__'): return f", from_f=from_f, f=f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
# Watch out: using \\n instead of \n inside a block
srcline = """
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
"""
dbcode = """
\n        dbprint("        from_f = getattr(from_f,'__func__',from_f)\\n\        to_f = getattr(to_f,'__func__',to_f)",\
"This line is for classmethod, as it is not callable, so inspect.signature(...) won't work, but it has __func__ to save",\
"f", "from_f", "type(from_f)",\
"try:\\n\
    inspect.signature(from_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is from_f callable: {callable(from_f)}')\\n\
else:\\n\
    print(inspect.signature(from_f))", \
"hasattr(from_f, '__func__')", "from_f = getattr(from_f,'__func__',from_f)", "from_f",\
"to", "to_f", "type(to_f)",\
"try:\\n\
    inspect.signature(to_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is to_f callable: {callable(to_f)}')\\n\
else:\\n\
    print(inspect.signature(to_f))", \
"hasattr(to_f, '__func__')", "to_f = getattr(to_f,'__func__',to_f)", "to_f",\
from_f=from_f, to_f=to_f, f=f, to=to, srcline=srcline)
"""
# Importance: must include all necessary env into the dbprint function above. Note, we don't need inspect=inspect because utils has exported `import inspect`

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
for s in srcdbps:
    print(s[0][0])


        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f


        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}

        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}

        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f

        if hasattr(from_f,'__delwrap__'): return f

        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)



In [ ]:

def matchsrcorder(srcdbps:list # the list contain all srclines and their dbcodes with random order
                 ):
    "Match srcdbps list in the same order as the official source code."
    srcdbps1 = [] # a list to store the correct order of srclines and dbcodes
    for l in defaults.src.split("\n"):
        for idx, s in zip(range(len(srcdbps)), srcdbps):
            if l.strip() in s[0][0]:
                srcdbps.pop(idx)
                srcdbps1.append(s)  
                
    defaults.src2dbp.delegates = srcdbps1
    return srcdbps1

In [ ]:
#|export
def matchsrcorder():
    "Match srcdbps list in the same order as the official source code."
    srcdbps = defaults.srcdbps
    srcdbps1 = [] # a list to store the correct order of srclines and dbcodes
    for l in defaults.src.split("\n"):
        for idx, s in zip(range(len(srcdbps)), srcdbps):
            if l.strip() in s[0][0]:
                srcdbps.pop(idx)
                srcdbps1.append(s)  
                
    defaults.src2dbp.delegates = srcdbps1
    return srcdbps1

In [ ]:
srcdbps = matchsrcorder()

In [ ]:
for s in srcdbps:
    print(s[0][0])

## display official source code with index markers

In [ ]:

def displaysrc(startsrc, endsrc):
    idx = 0
    mark = False
    for l in defaults.src.split("\n"):

        if startsrc in l: mark = True


        if mark:
            marker = f'( {idx} )' + "     "
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)


In [ ]:

def displaysrc(srcname:str, # name of src code like delegates
               startsrc:str, # a piece of code like "if to is None"
               endsrc:str): # a piece of code like "from_f.__annotations__.update(anno)"
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    srclines = "" # store all the debuggable srclines here
    for i in srcdblist:
        srclines = srclines + i[0][0]
    
    idx = 0
    mark = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            if l in srclines:
                marker = f'( {idx} )' + "====="
            else:
                marker = f'( {idx} )' + "     "
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

### add index marker for debuggable source lines

In [ ]:

def displaysrc():
    srcname = defaults.name # name of src code like delegates
    startsrc = defaults.startsrc # a piece of code like "if to is None"
    endsrc = defaults.endsrc # a piece of code like "from_f.__annotations__.update(anno)"
    
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    srclines = "" # store all the debuggable srclines here
    for i in srcdblist:
        srclines = srclines + i[0][0]
    
    idx = 0
    mark = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            if l in srclines:
                marker = f'( {idx} )' + "====="
            else:
                marker = f'( {idx} )' + "     "
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

### Marking the debuggable srcline with its own index

In [ ]:

def displaysrc():
    "display the official source code also marking the debuggable srclines"
    srcname = defaults.name # name of src code like delegates
    startsrc = defaults.startsrc # a piece of code like "if to is None"
    endsrc = defaults.endsrc # a piece of code like "from_f.__annotations__.update(anno)"
    
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    idx = 0
    mark = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            marker = f'( {idx} )' + "     "
            for idxi, i in zip(range(len(srcdblist)), srcdblist):

                if l.strip() in i[0][0]:
                    premark = f'=========='
                    marker = premark + f'( {idx} )' + f'=={idxi}=='
                    continue
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

In [ ]:

def displaysrc():
    "display the official source code also marking the debuggable srclines"
    srcname = defaults.name # name of src code like delegates
    startsrc = defaults.startsrc # a piece of code like "if to is None"
    endsrc = defaults.endsrc # a piece of code like "from_f.__annotations__.update(anno)"
    
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    idx = 0
    mark = False
    passl = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            marker = f'( {idx} )' + "     "
            for idxi, i in zip(range(len(srcdblist)), srcdblist):

                if l.strip() in i[0][0]:

                    marker = f'( {idx} )' + f'=={idxi}=='
                    indent = defaults.margin - len(l) - len(marker)
                    print(l + "="*indent + marker)
                    passl = True
                    continue # jump out of the inner for loop

            if passl: # make sure to jump out of the outer for loop
                passl = False
                idx = idx + 1 # keep track idx for every srcline to be debugged
                continue
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

In [ ]:

def displaysrc():
    "display the official source code also marking the debuggable srclines. behind the scene, loading defaults.src2dbp.{srcname} is loaded from a pickle file if available."
    srcname = defaults.name # name of src code like delegates
    startsrc = defaults.startsrc # a piece of code like "if to is None"
    endsrc = defaults.endsrc # a piece of code like "from_f.__annotations__.update(anno)"
    
    
    if exists(f"Users/Natsume/Documents/debuggable/fastcore/meta/db/{srcname}"):
        with open(f"db/{srcname}", "rb") as fp:   
          exec(f"defaults.src2dbp.{srcname} = pickle.load(fp)")
    else:
        pass
    
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    idx = 0
    mark = False
    passl = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            marker = f'( {idx} )' + "     "
            for idxi, i in zip(range(len(srcdblist)), srcdblist):

                if l.strip() in i[0][0]:

                    marker = f'( {idx} )' + f'=={idxi}=='
                    indent = defaults.margin - len(l) - len(marker)
                    print(l + "="*indent + marker)
                    passl = True
                    continue # jump out of the inner for loop

            if passl: # make sure to jump out of the outer for loop
                passl = False
                idx = idx + 1 # keep track idx for every srcline to be debugged
                continue
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

In [ ]:
#|export
from pathlib import Path

In [ ]:

# path = Path(f"db/{defaults.name}")

# if path.is_file():
#     with open("db/delegates", "rb") as fp:   # Unpickling
#         exec(f"defaults.src2dbp.{defaults.name} = pickle.load(fp)")
# else:
#     pass


In [ ]:
# with open("db/delegates", "rb") as fp:   # Unpickling
#   exec(f"defaults.src2dbp.{defaults.name} = pickle.load(fp)")

In [ ]:
#|export
def displaysrc():
    "display the official source code also marking the debuggable srclines. behind the scene, loading defaults.src2dbp.{srcname} is loaded from a pickle file if available."
    srcname = defaults.name # name of src code like delegates
    startsrc = defaults.startsrc # a piece of code like "if to is None"
    endsrc = defaults.endsrc # a piece of code like "from_f.__annotations__.update(anno)"
    
    
    path = Path(f"db/{srcname}")

    
    if path.is_file():
        with open(f"db/{srcname}", "rb") as fp:   # Unpickling
            exec(f"defaults.src2dbp.{srcname} = pickle.load(fp)")
    else:
        print(f"Warning: there is no such file named db/{srcname}, so defaults.src2dbp.{srcname} is empty.")
        pass

    
    srcdblist = eval("defaults.src2dbp." + srcname)
    srcdblist = L(srcdblist)
    
    idx = 0
    mark = False
    passl = False
    for l in defaults.src.split("\n"):

        # to mark the index for the targed src codes
        if startsrc in l: mark = True
        if mark:
            marker = f'( {idx} )' + "     "
            for idxi, i in zip(range(len(srcdblist)), srcdblist):

                if l.strip() in i[0][0]:

                    marker = f'( {idx} )' + f'=={idxi}=='
                    indent = defaults.margin - len(l) - len(marker)
                    print(l + "="*indent + marker)
                    passl = True
                    continue # jump out of the inner for loop

            if passl: # make sure to jump out of the outer for loop
                passl = False
                idx = idx + 1 # keep track idx for every srcline to be debugged
                continue
            indent = defaults.margin - len(l) - len(marker)
            print(l + " "*indent + marker)
            idx = idx + 1
            if endsrc in l: mark = False
        else:
            print(l)

#|hide
## Export

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()

#|hide
## Sending to Obs

In [ ]:
#|hide
!jupytext --to md /Users/Natsume/Documents/debuggable/utils.ipynb
!mv /Users/Natsume/Documents/debuggable/utils.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/

[jupytext] Reading /Users/Natsume/Documents/debuggable/utils.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/debuggable/utils.md


In [ ]:
#|hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/index.ipynb to markdown
[NbConvertApp] Writing 2074 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/utils.ipynb to markdown
[NbConvertApp] Writing 209841 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/utils.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/classes_metaclasses.ipynb to markdown
[NbConvertApp] Writing 23234 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/classes_metaclasses.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/00_FixSigMeta.ipynb to markdown
[NbConvertApp] Writing 66757 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/00_FixSigMeta.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/02_use_kwargs_dict.ipynb to markdown
[NbConvertApp] 

In [ ]:
_signature_from_callable

<function inspect._signature_from_callable(obj, *, follow_wrapper_chains=True, skip_bound_arg=True, sigcls)>